In [1]:
# # #Download ctrNet-tool 
# # #You can find the code in https://github.com/guoday/ctrNet-tool
!git clone https://github.com/guoday/ctrNet-tool.git
!cp -r ctrNet-tool/* ./
!rm -r ctrNet-tool data .git
!ls -all

Cloning into 'ctrNet-tool'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 168 (delta 86), reused 168 (delta 86), pack-reused 0
Receiving objects: 100% (168/168), 8.42 MiB | 0 bytes/s, done.
Resolving deltas: 100% (86/86), done.
rm: cannot remove '.git': No such file or directory
total 44
drwxr-xr-x 4 root root 4096 Apr  4 21:31 .
drwxr-xr-x 6 root root 4096 Apr  4 21:31 ..
-rw-r--r-- 1 root root 2402 Apr  4 21:31 README.md
-rw-r--r-- 1 root root 6170 Apr  4 21:31 __notebook__.ipynb
-rw-r--r-- 1 root root  271 Apr  4 21:31 __output__.json
-rw-r--r-- 1 root root  772 Apr  4 21:31 ctrNet.py
drwxr-xr-x 2 root root 4096 Apr  4 21:31 models
drwxr-xr-x 2 root root 4096 Apr  4 21:31 src
-rw-r--r-- 1 root root 5225 Apr  4 21:31 test.py


In [2]:
import ctrNet
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from src import misc_utils as utils
import os
import gc
import random

In [3]:
print('Loading Train and Test Data.\n')
# train = pd.read_csv('../input/malware-yannis2/freq_train.csv', low_memory=True)
# test  = pd.read_csv('../input/malware-yannis2/freq_test.csv', low_memory=True)
train = pd.read_csv('../input/malware-ffe/train_ffe.csv', low_memory=True)
test  = pd.read_csv('../input/malware-ffe/test_ffe.csv', low_memory=True)

Loading Train and Test Data.



/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
for c in ['index', 'MachineIdentifier']:
    if  c in train.columns:
        train.drop(c, axis=1, inplace=True); # = train.index.astype('uint32')
        test.drop(c, axis=1, inplace=True); #  = test.index.astype('uint32')

In [5]:
train.shape

(2230371, 102)

In [6]:
test.shape

(334556, 102)

In [7]:
train.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,IeVerIdentifier,SmartScreen,Firewall,Census_MDC2FormFactor,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,...,Wdft_IsGamer,Wdft_RegionIdentifier,OsBuildReleaseYearMonth,AppVersion2,Lag1,Lag5,driveA,driveB,Census_OSVersion_FE,Census_OSBuildRevision_FE,Census_InternalBatteryNumberOfCharges_FE,AvSigVersion_FE,Lag1_FE,CountryIdentifier_FE,HasDetections,SmartScreen_BE_ExistsNotSet,CountryIdentifier_BE_89_214,Processor_BE_x86,Census_IsTouchEnabled_BE_1,Census_HasOpticalDiskDrive_BE_1,Census_InternalPrimaryDisplayResolutionVertical_BE_800_600,Census_PowerPlatformRoleName_BE_Slate,Wdft_IsGamer_BE_1,Census_MDC2FormFactor_BE_Detachable_LargeTablet,Census_OSWUAutoUpdateOptionsName_BE_FullAuto,AVProductsInstalled_FE,Census_OSInstallTypeName_FE,OsBuild_FE,EngineVersion_FE,OsPlatformSubRelease_FE,IeVerIdentifier_FE,Firewall_FE,Census_PrimaryDiskTypeName_FE,Census_TotalPhysicalRAM_FE,Census_OSBranch_FE,Census_OSBuildNumber_FE,Census_FlightRing_FE,Census_FirmwareManufacturerIdentifier_FE,Wdft_RegionIdentifier_FE,Census_ProcessorModelIdentifier_FE
0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.94.0,7.0,0,0.0,53447.0,1.0,1.0,1,86,Empty,18.0,98,103,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,107.0,NoExist,1.0,Notebook,2668.0,171357.0,4.0,5.0,2382.0,953869.0,HDD,913759.0,0,4096.0,Notebook,15.5,...,0.0,3.0,1703.0,18,40.0,0.0,0.957950,40110.0,0.029215,0.029855,0.574588,0.001036,0.002760,0.104395,0,0,0,0,0,0,0,0,0,0,0,1.0,0.310790,0.228605,0.316451,0.228605,0.041562,1.0,1.000000,1.000000,0.227815,0.227823,1.00000,0.463933,0.657560,0.595175
1,win8defender,1.1.15200.1,4.18.1807.18075,1.275.59.0,7.0,0,0.0,53447.0,1.0,1.0,1,201,45572.0,27.0,267,251,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,137.0,NoExist,1.0,Desktop,3035.0,263668.0,4.0,5.0,3478.0,114473.0,SSD,113857.0,0,8192.0,Desktop,21.5,...,1.0,11.0,1804.0,18,1.0,0.0,0.994619,616.0,0.941126,0.941132,0.244710,0.002372,0.089295,0.552062,0,0,0,0,0,0,0,0,0,0,1,1.0,0.222675,1.000000,0.316451,1.000000,1.000000,1.0,0.397153,0.514310,1.000000,1.000000,1.00000,1.000000,0.713265,0.052183
2,win8defender,1.1.15200.1,4.18.1807.18075,1.275.180.0,7.0,0,0.0,53447.0,1.0,1.0,1,171,96401.0,27.0,211,182,x64,10.0.0.0,16299,256,rs3,16299.15.amd64fre.rs3_release.170928-1534,Pro,1.0,117.0,RequireAdmin,1.0,Desktop,1980.0,224748.0,3.0,1.0,481.0,476940.0,HDD,476477.0,0,6144.0,Desktop,21.5,...,0.0,3.0,1709.0,18,19.0,0.0,0.999029,463.0,0.286985,0.286985,0.244710,0.001020,0.013140,0.691985,1,0,0,0,0,0,0,0,0,0,0,1.0,0.464936,0.747548,0.316451,0.747548,0.534373,1.0,1.000000,0.097366,0.350734,0.714349,1.00000,0.029058,0.657560,0.003314
3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.463.0,7.0,0,0.0,53447.0,1.0,1.0,1,81,66998.0,0.0,107,224,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,137.0,RequireAdmin,1.0,Notebook,1443.0,256567.0,4.0,5.0,2574.0,476940.0,HDD,463362.0,1,4096.0,Laptop,13.9,...,1.0,10.0,1804.0,18,2.0,0.0,0.971531,13578.0,0.941126,0.941132,0.574588,0.000529,0.102115,0.110950,1,0,0,0,0,0,0,0,0,0,1,1.0,1.000000,1.000000,0.316451,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.00000,0.353502,1.000000,0.364916
4,win8defender,1.1.14104.0,4.12.16299.15,1.251.42.0,7.0,0,239.0,53447.0,1.0,1.0,1,89,164840.0,0.0,120,262,x64,10.0.0.0,16299,256,rs

In [8]:
def make_bucket(data,num=10):
    data.sort()
    bins=[]
    for i in range(num):
        bins.append(data[int(len(data)*(i+1)//num)-1])
    return bins
float_features=['Census_SystemVolumeTotalCapacity','Census_PrimaryDiskTotalCapacity']
for f in float_features:
    train[f]=train[f].fillna(1e10)
    test[f]=test[f].fillna(1e10)
    data=list(train[f])+list(test[f])
    bins=make_bucket(data,num=50)
    train[f]=np.digitize(train[f],bins=bins)
    test[f]=np.digitize(test[f],bins=bins)

In [9]:
features=train.columns.tolist()
features.remove('HasDetections')

hparam=tf.contrib.training.HParams(
            model='xdeepfm',
            norm=True,
            batch_norm_decay=0.9,
            hidden_size=[128,128],
            cross_layer_sizes=[128,128,128],
            k=8,
            hash_ids=int(2e5),
            batch_size=1024,
            optimizer="adam",
            learning_rate=0.001,
            num_display_steps=1000,
            num_eval_steps=1000,
            epoch=1,
            metric='auc',
            activation=['relu','relu','relu'],
            cross_activation='identity',
            init_method='uniform',
            init_value=0.1,
            feature_nums=len(features),
            kfold=5)
# utils.print_hparams(hparam)

In [10]:
import copy 

dev_acc = []
dev_auc = []
best_dev_auc = 0
for i in range(3):
    train_sub, dev,_,_ = train_test_split(train,train['HasDetections'],test_size=0.1, random_state=2019+i)
    model=ctrNet.build_model(hparam)
    train_data = train_sub[features]
    train_labels = train_sub['HasDetections']
    dev_data = dev[features]
    dev_labels = dev['HasDetections']
    print('Length of train_data ', train_data.shape, 'train labels', train_labels.shape)
    print('Length of dev_data ', dev_data.shape, 'train labels', dev_labels.shape)
    model.train(train_data=(train_data, train_labels),\
                dev_data=(dev_data, dev_labels))
    preds=model.infer(dev_data=(dev_data, dev_labels))
    dev_acc_latest = metrics.accuracy_score(dev_labels, preds>0.5)
    fpr, tpr, thresholds = metrics.roc_curve(dev_labels, preds)
    dev_auc_latest=metrics.auc(fpr, tpr)
    dev_acc.append(dev_acc_latest)
    dev_auc.append(dev_auc_latest)
    if dev_auc_latest > best_dev_auc:
        best_dev_auc = dev_auc_latest
        best_model = model
    print('Dev accuracy: ', np.mean(dev_acc))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
# Trainable variables
  emb_v1:0, (200000, 1), 
  emb_v2:0, (200000, 8), 
  Variable:0, (808, 128), 
  norm_0/beta:0, (128,), 
  norm_0/gamma:0, (128,), 
  Variable_1:0, (128, 128), 
  norm_1/beta:0, (128,), 
  norm_1/gamma:0, (128,), 
  Variable_2:0, (128, 1), 
  exfm_part/f_0:0, (1, 10201, 128), 
  exfm_part/f_1:0, (1, 6464, 128), 
  exfm_part/f_2:0, (1, 6464, 128), 
  exfm_part/w_nn_output:0, (256, 1), 
  exfm_part/b_nn_output:0, (1,), 
Length of train_data  (2007333, 101) train labels (2007333,)
Length of dev_data  (223038, 101) train labels (223038,)
  epoch 0 step 1000 lr 0.001 logloss 0.613543 gN 0.28, Thu Apr  4 21:35:30 2019
# Epcho-time 187.08s Eval AUC 0.731056. Best AUC 0.731056.
# Epcho-time 400.28s Eval AUC 0.738403. Best AUC 0.738403.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring paramet

In [11]:
test_data = test[features]
test_labels = test['HasDetections']
preds=best_model.infer(dev_data=(test_data, test_labels))
test_acc = metrics.accuracy_score(test_labels, preds>0.5)
print(test_acc)
fpr, tpr, thresholds = metrics.roc_curve(test_labels, preds)
test_auc=metrics.auc(fpr, tpr)

import json
results = {'dev_acc': dev_acc,
           'dev_auc': dev_auc,
           'test_acc': test_acc,
          'test_auc': test_auc}
with open('results.out', 'w') as f:
    json.dump(results, f)

0.6253332775379906


In [12]:
cat results.out

{"dev_acc": [0.6693747253831186, 0.6705449295635721, 0.6688949865045418], "dev_auc": [0.7384044964609742, 0.7395131049227297, 0.7377821861054451], "test_acc": 0.6253332775379906, "test_auc": 0.6815512643603836}

In [13]:
# 1 epoch
# {"dev_acc": [0.6694643962015441, 0.6690339762731015, 0.6692581533191654], 
# "dev_auc": [0.7381366056194263, 0.7369799404430174, 0.7376166835572143], 
#  "test_acc": 0.6336308420712825, 
# "test_auc": 0.6906173348280547}